In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install bertopic

env: TOKENIZERS_PARALLELISM=false
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 49.1 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819467 sha256=3dfeff713a72d7908ba40d31887c7836450036d94826a143f64873c56814f44b
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=2ac0f9c949948e7895c172a3eca771191e030f9bfda87e1497a082eabe496dd4
  Stored in directory: /root/.cache/pip/whee

In [2]:
import pandas as pd
USECOLS = ['description', 'category', 'family', 'mechanic', 'game_id']
df = pd.read_csv(filepath_or_buffer='/kaggle/input/board-games/board_games.csv', index_col=['game_id'], usecols=USECOLS)
df['text'] = df.apply(axis=1, func=lambda x: ','.join(x.dropna().astype(str)),)
df.head()

,description,category,family,mechanic,text
game_id,,,,,
1,Die Macher is a game about seven sequential po...,"Economic,Negotiation,Political","Country: Germany,Valley Games Classic Line","Area Control / Area Influence,Auction/Bidding,...",Die Macher is a game about seven sequential po...
2,Dragonmaster is a trick-taking card game based...,"Card Game,Fantasy",Animals: Dragons,Trick-taking,Dragonmaster is a trick-taking card game based...
3,"Part of the Knizia tile-laying trilogy, Samura...","Abstract Strategy,Medieval","Asian Theme,Country: Japan,Knizia tile-laying ...","Area Control / Area Influence,Hand Management,...","Part of the Knizia tile-laying trilogy, Samura..."
4,When you see the triangular box and the luxuri...,Ancient,"Country: Egypt,Promotional Board Games","Action Point Allowance System,Area Control / A...",When you see the triangular box and the luxuri...
5,"In Acquire, each player strategically invests ...",Economic,3M Bookshelf Series,"Hand Management,Stock Holding,Tile Placement","In Acquire, each player strategically invests ..."


In [3]:
from arrow import now
from bertopic import BERTopic
from plotly.express import histogram
from sklearn.feature_extraction.text import CountVectorizer

SAMPLE_FRACTION = 1.0
time_start = now()
large_model = BERTopic(verbose=True, top_n_words=25, nr_topics=None, language='english', 
                       vectorizer_model=CountVectorizer(stop_words='english', lowercase=True))
large_topics, large_probs = large_model.fit_transform(df['text'].sample(frac=SAMPLE_FRACTION, random_state=2023).dropna().values)
print('After {} our topic says we have {} topics.'.format(now() - time_start, len(set(large_topics))))
print('And we have {} documents with no topic ({}%).'.format(large_topics.count(-1), round(100 * large_topics.count(-1)/len(large_topics))))
histogram(x=[item for item in large_topics if item > -1], title = 'topic > -1 histogram')

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

Batches:   0%|          | 0/330 [00:00<?, ?it/s]

2023-10-22 14:01:50,172 - BERTopic - Transformed documents to Embeddings
2023-10-22 14:02:31,509 - BERTopic - Reduced dimensionality
2023-10-22 14:02:32,105 - BERTopic - Clustered reduced embeddings


After 0:17:18.965816 our topic says we have 157 topics.
And we have 4022 documents with no topic (38%).


In [4]:
large_model.visualize_topics(width=1000, height=1000)